In [19]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
import joblib
from flask import Flask, jsonify, request

In [20]:
# create Flask app
app = Flask(__name__)

In [21]:
column_categories = joblib.load('column_categories.gz')
model = joblib.load('model.gz')

In [22]:
# helper function
def apply_categories(df, column_categories):
    for col, categories in column_categories.items():
        df[col] = df[col].astype(CategoricalDtype(categories=categories))
    return df

In [23]:
# create API endpoint
@app.route('/api', methods=['POST'])
def predict():
    # read the JSON received
    data = request.get_json()
    # transform it into a DataFrame
    df = pd.DataFrame(data, index=[0])
    # transform categorical columns
    df = apply_categories(df, column_categories)
    df = pd.get_dummies(df, column_categories.keys())
    # predict
    prediction = model.predict(df)
    # convert the prediction from a numpy array to a string
    str_pred = np.array2string(prediction)
    # convert it to JSON
    return jsonify(str_pred)

Test request

In [24]:
import socket
import threading
import requests

In [25]:
 # Create a new thread for running Flask app
flask_thread = threading.Thread(target=app.run, kwargs={'host':'0.0.0.0','port':80, 'debug': False})
flask_thread.start()

 * Serving Flask app &quot;__main__&quot; (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


In [26]:
X_test = joblib.load('X_test.gz')
y_test = joblib.load('y_test.gz')

In [27]:
def test_predict(json_data):
    import json
    # read the JSON received
    data = json.loads(json_data)
    # transform it into a DataFrame
    df = pd.DataFrame(data, index=[0])
    # transform categorical columns
    df = apply_categories(df, column_categories)
    df = pd.get_dummies(df, column_categories.keys())
    # predict
    prediction = model.predict(df)
    # convert the prediction from a numpy array to a string
    str_pred = np.array2string(prediction)
    # convert it to JSON
    return str_pred

In [28]:
# test record
record = X_test.iloc[0, :].to_json() 
r = test_predict(record)
print(r)
print(y_test.iloc[0])

[&#39; &lt;=50K&#39;]
 &lt;=50K


In [30]:
# test record
record = X_test.iloc[0, :].to_json() 
print(record)
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
ip_address = socket.gethostbyname(socket.gethostname())
r = requests.post(f"http://{ip_address}/api", data=record, headers=headers)
print(r.text)
print(y_test.iloc[0])

{&quot;age&quot;:52,&quot;workclass&quot;:&quot; Private&quot;,&quot;fnlwgt&quot;:117700,&quot;education&quot;:&quot; HS-grad&quot;,&quot;education-num&quot;:9,&quot;marital-status&quot;:&quot; Divorced&quot;,&quot;occupation&quot;:&quot; Adm-clerical&quot;,&quot;relationship&quot;:&quot; Not-in-family&quot;,&quot;sex&quot;:&quot; Female&quot;,&quot;capital-gain&quot;:0,&quot;capital-loss&quot;:0,&quot;hours-per-week&quot;:40,&quot;native-country&quot;:&quot; United-States&quot;}
&lt;!DOCTYPE HTML PUBLIC &quot;-//W3C//DTD HTML 3.2 Final//EN&quot;&gt;
&lt;title&gt;500 Internal Server Error&lt;/title&gt;
&lt;h1&gt;Internal Server Error&lt;/h1&gt;
&lt;p&gt;The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.&lt;/p&gt;

 &lt;=50K
